# REDES NEURAIS ARTIFICIAIS
## Evaluate Improve e Tunning

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.8.0'

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [5]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print('X>>\n', X)

X>>
 [[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [6]:
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print('X>>\n', X)

X>>
 [[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [10]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [11]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [12]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [13]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [14]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [15]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 797us/step - loss: 0.6633 - accuracy: 0.6864
Epoch 2/100
250/250 [==============================] - 0s 734us/step - loss: 0.5004 - accuracy: 0.7966
Epoch 3/100
250/250 [==============================] - 0s 616us/step - loss: 0.4426 - accuracy: 0.8094
Epoch 4/100
250/250 [==============================] - 0s 697us/step - loss: 0.4256 - accuracy: 0.8150
Epoch 5/100
250/250 [==============================] - 0s 767us/step - loss: 0.4155 - accuracy: 0.8216
Epoch 6/100
250/250 [==============================] - 0s 744us/step - loss: 0.4034 - accuracy: 0.8286
Epoch 7/100
250/250 [==============================] - 0s 695us/step - loss: 0.3899 - accuracy: 0.8378
Epoch 8/100
250/250 [==============================] - 0s 595us/step - loss: 0.3771 - accuracy: 0.8431
Epoch 9/100
250/250 [==============================] - 0s 543us/step - loss: 0.3667 - accuracy: 0.8481
Epoch 10/100
250/250 [==============================] - 0s 545us/step - l

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [16]:
y_pred = ann.predict(X_test)
y_pred

array([[0.21772182],
       [0.27619475],
       [0.13028958],
       ...,
       [0.09519646],
       [0.14674127],
       [0.20815203]], dtype=float32)

In [17]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [19]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1528   67]
 [ 206  199]]


---
# Avaliando o Modelo

In [20]:
# Evaluating the ANN
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [22]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [23]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

<ipython-input-23-b2917738c23b>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)


In [25]:
print(mean,variance)

0.8403750061988831 0.013829881795730697


# Melhorando o modelo

In [26]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed
from tensorflow.keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [27]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

<ipython-input-27-b2917738c23b>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)


In [28]:
print(mean,variance)

0.8211250007152557 0.015180681964707187


# Tuning the ANN

In [29]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 25, 32],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop']
}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

<ipython-input-29-564c6c36d575>:16: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_classifier)


In [30]:
%%time

grid_search = grid_search.fit(X_train, y_train)

Epoch 1/50
720/720 [==============================] - 1s 753us/step - loss: 0.4947 - accuracy: 0.7971
Epoch 2/50
720/720 [==============================] - 0s 567us/step - loss: 0.4362 - accuracy: 0.7971
Epoch 3/50
720/720 [==============================] - 0s 602us/step - loss: 0.4316 - accuracy: 0.7971
Epoch 4/50
720/720 [==============================] - 1s 725us/step - loss: 0.4280 - accuracy: 0.7971
Epoch 5/50
720/720 [==============================] - 0s 611us/step - loss: 0.4271 - accuracy: 0.7971
Epoch 6/50
720/720 [==============================] - 0s 634us/step - loss: 0.4291 - accuracy: 0.8133
Epoch 7/50
720/720 [==============================] - 0s 560us/step - loss: 0.4234 - accuracy: 0.8249
Epoch 8/50
720/720 [==============================] - 0s 535us/step - loss: 0.4264 - accuracy: 0.8271
Epoch 9/50
720/720 [==============================] - 0s 608us/step - loss: 0.4234 - accuracy: 0.8257
Epoch 10/50
720/720 [==============================] - 0s 642us/step - loss: 0.423

KeyboardInterrupt: 

In [31]:
print('Best params:\n',grid_search.best_params_,'\nBest accuracy:\n',grid_search.best_score_)

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

# Halving Grid Search

In [37]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

def build_classifier(optimizer, activation, init):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = init, activation = activation, input_dim = 12))
#     classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = init, activation = activation))
#     classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 50],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop'],
    'activation': ['softmax', 'relu', 'tanh'],
    'init': ['uniform', 'normal', 'zero']
}

halving_cv = HalvingGridSearchCV(estimator = classifier,
                                  param_grid = parameters,
                                  scoring="accuracy", 
                                  n_jobs=-1, 
                                  min_resources="exhaust", 
                                  factor=3,
                                  cv = 10)

<ipython-input-37-225b1b7f08e4>:19: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_classifier)


In [39]:
%%time
grid_result = halving_cv.fit(X_train, y_train)

Epoch 1/100
800/800 [==============================] - 1s 648us/step - loss: 0.4848 - accuracy: 0.8079
Epoch 2/100
800/800 [==============================] - 1s 660us/step - loss: 0.4355 - accuracy: 0.8138
Epoch 3/100
800/800 [==============================] - 1s 700us/step - loss: 0.4332 - accuracy: 0.8117
Epoch 4/100
800/800 [==============================] - 1s 640us/step - loss: 0.4263 - accuracy: 0.8144
Epoch 5/100
800/800 [==============================] - 1s 694us/step - loss: 0.4180 - accuracy: 0.8181
Epoch 6/100
800/800 [==============================] - 1s 625us/step - loss: 0.4048 - accuracy: 0.8275
Epoch 7/100
800/800 [==============================] - 0s 560us/step - loss: 0.3980 - accuracy: 0.8322
Epoch 8/100
800/800 [==============================] - 0s 587us/step - loss: 0.3950 - accuracy: 0.8321
Epoch 9/100
800/800 [==============================] - 1s 710us/step - loss: 0.3968 - accuracy: 0.8314
Epoch 10/100
800/800 [==============================] - 1s 711us/step - l

In [40]:
print('Best params:\n',grid_result.best_params_,'\nBest accuracy:\n',grid_result.best_score_)

Best params:
 {'activation': 'tanh', 'batch_size': 10, 'epochs': 100, 'init': 'normal', 'optimizer': 'adam'} 
Best accuracy:
 0.8546933667083856


In [41]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mena, std, param in zip(means, stds, params):
    print('{},{} with: {}', mean, std, param)

{},{} with: {} 0.8211250007152557 0.06254605912833466 {'activation': 'softmax', 'batch_size': 10, 'epochs': 50, 'init': 'uniform', 'optimizer': 'adam'}
{},{} with: {} 0.8211250007152557 0.06254605912833466 {'activation': 'softmax', 'batch_size': 10, 'epochs': 50, 'init': 'uniform', 'optimizer': 'rmsprop'}
{},{} with: {} 0.8211250007152557 0.06254605912833466 {'activation': 'softmax', 'batch_size': 10, 'epochs': 50, 'init': 'normal', 'optimizer': 'adam'}
{},{} with: {} 0.8211250007152557 0.06254605912833466 {'activation': 'softmax', 'batch_size': 10, 'epochs': 50, 'init': 'normal', 'optimizer': 'rmsprop'}
{},{} with: {} 0.8211250007152557 0.06254605912833466 {'activation': 'softmax', 'batch_size': 10, 'epochs': 50, 'init': 'zero', 'optimizer': 'adam'}
{},{} with: {} 0.8211250007152557 0.06254605912833466 {'activation': 'softmax', 'batch_size': 10, 'epochs': 50, 'init': 'zero', 'optimizer': 'rmsprop'}
{},{} with: {} 0.8211250007152557 0.06254605912833466 {'activation': 'softmax', 'batch_

In [42]:
grid_result.cv_results_.keys()

dict_keys(['iter', 'n_resources', 'mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_activation', 'param_batch_size', 'param_epochs', 'param_init', 'param_optimizer', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'split5_test_score', 'split6_test_score', 'split7_test_score', 'split8_test_score', 'split9_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score', 'split3_train_score', 'split4_train_score', 'split5_train_score', 'split6_train_score', 'split7_train_score', 'split8_train_score', 'split9_train_score', 'mean_train_score', 'std_train_score'])

In [46]:
dir(grid_result)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_input_parameters',
 '_check_is_fitted',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_checked_cv_orig',
 '_estimator_type',
 '_format_results',
 '_generate_candidate_params',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_n_samples_orig',
 '_pairwise',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_validate_data',
 'aggressive_elimination',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'err

# FIM

> Aluno Nitai Bezerra
> [@nitaibezerra](https://twitter.com/nitaibezerra)